# Polynomial Overfitting Example (Section 7.1)

**The Problem**: More parameters always improve fit to training data, but this doesn't mean better predictions!

**Example**: Primate brain volume vs body mass
- 7 species (small dataset!)
- Fit polynomials of degree 1, 2, 3, 4, 5, 6
- Watch R² improve while models become absurd

**Key Lesson**: We need measures beyond R² to evaluate models!

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent.parent))
from src.quap import quap

plt.style.use('default')
%matplotlib inline

np.random.seed(42)

print('✓ Imports loaded')

## Load Primate Brain Data

From Statistical Rethinking - brain volume and body mass for 7 primate species.

In [ ]:
# Load the data
# Note: This is a small dataset from the book
# Species: Different primates
# brain: Brain volume (cc)
# mass: Body mass (kg)

# Create the dataset (from the book)
data = pd.DataFrame({
    'species': ['afarensis', 'africanus', 'habilis', 'boisei',
                'rudolfensis', 'ergaster', 'sapiens'],
    'brain': [438, 452, 612, 521, 752, 871, 1350],
    'mass': [37.0, 35.5, 34.5, 41.5, 55.5, 61.0, 53.5]
})

print(f"Dataset: {len(data)} primate species")
print("\nData:")
data

In [ ]:
# Standardize the variables
brain_std = (data['brain'] - data['brain'].mean()) / data['brain'].std()
mass_std = (data['mass'] - data['mass'].mean()) / data['mass'].std()

print("Variables standardized")
print(f"Brain (standardized): mean={brain_std.mean():.2f}, std={brain_std.std():.2f}")
print(f"Mass (standardized): mean={mass_std.mean():.2f}, std={mass_std.std():.2f}")

## Fit Polynomial Models of Increasing Degree

We'll fit 6 models:
1. **Linear** (degree 1): μ = α + β₁x
2. **Quadratic** (degree 2): μ = α + β₁x + β₂x²
3. **Cubic** (degree 3): μ = α + β₁x + β₂x² + β₃x³
4. **Quartic** (degree 4): μ = α + β₁x + ... + β₄x⁴
5. **Quintic** (degree 5): μ = α + β₁x + ... + β₅x⁵
6. **Sextic** (degree 6): μ = α + β₁x + ... + β₆x⁶

**Prediction**: R² will improve with each degree, but models will become absurd!

In [ ]:
# Function to fit polynomial of given degree
def fit_polynomial(x, y, degree):
    """
    Fit a polynomial regression of given degree using quap.
    
    Model: y ~ Normal(μ, σ)
           μ = α + β₁x + β₂x² + ... + βₙxⁿ
    """
    # Create polynomial features
    X_poly = np.column_stack([x**i for i in range(1, degree + 1)])
    
    # Define negative log posterior
    def neg_log_posterior(params):
        alpha = params[0]
        betas = params[1:degree+1]
        log_sigma = params[degree+1]
        sigma = np.exp(log_sigma)
        
        # Linear predictor
        mu = alpha + np.sum(X_poly * betas, axis=1)
        
        # Log likelihood
        log_lik = np.sum(stats.norm.logpdf(y, loc=mu, scale=sigma))
        
        # Priors
        log_prior = (stats.norm.logpdf(alpha, 0, 0.2) +
                    np.sum(stats.norm.logpdf(betas, 0, 0.5)) +
                    stats.expon.logpdf(sigma, scale=1))
        
        return -(log_lik + log_prior + log_sigma)
    
    # Initial parameters: alpha, beta1, ..., betan, log_sigma
    initial = [0] * (degree + 2)
    param_names = ['alpha'] + [f'beta{i}' for i in range(1, degree+1)] + ['log_sigma']
    
    # Fit with quap
    fit = quap(neg_log_posterior, initial, param_names)
    fit.transform_param('log_sigma', 'sigma', np.exp)
    
    return fit

print("✓ Polynomial fitting function defined")

In [ ]:
# Fit all 6 models
models = {}
degrees = range(1, 7)

for deg in degrees:
    print(f"\nFitting polynomial degree {deg}...")
    models[deg] = fit_polynomial(mass_std.values, brain_std.values, deg)
    print(f"  ✓ Converged: {models[deg].success}")

print("\n✓ All models fitted!")

## Compute R² for Each Model

**R²** measures proportion of variance explained.

**Watch**: R² will increase monotonically as we add parameters!

In [ ]:
# Compute R² for each model
def compute_r_squared(model, x, y, degree):
    """
    Compute R² for a fitted polynomial model.
    """
    # Get coefficients
    coefs = model.coef()
    alpha = coefs['alpha']
    betas = [coefs[f'beta{i}'] for i in range(1, degree+1)]
    
    # Predictions
    X_poly = np.column_stack([x**i for i in range(1, degree + 1)])
    y_pred = alpha + np.sum(X_poly * betas, axis=1)
    
    # R²
    ss_res = np.sum((y - y_pred)**2)
    ss_tot = np.sum((y - y.mean())**2)
    r_squared = 1 - (ss_res / ss_tot)
    
    return r_squared

# Compute R² for all models
r_squared_values = {}
for deg in degrees:
    r_squared_values[deg] = compute_r_squared(models[deg], 
                                              mass_std.values, 
                                              brain_std.values, 
                                              deg)

# Display results
results_df = pd.DataFrame({
    'Degree': list(degrees),
    'Parameters': [deg + 2 for deg in degrees],  # +2 for alpha and sigma
    'R²': [r_squared_values[deg] for deg in degrees]
})

print("\nModel Comparison (by R²):")
print("="*50)
print(results_df.to_string(index=False))
print("="*50)
print("\n⚠️ Notice: R² increases monotonically!")
print("   But are higher degree models really better?")

## Visualize the Fits - Recreate Figure 7.1

Let's plot all 6 polynomial fits to see how they become increasingly wiggly and absurd!

In [ ]:
# Create predictions for smooth curves
x_plot = np.linspace(mass_std.min() - 0.5, mass_std.max() + 0.5, 100)

# Function to generate predictions
def predict_polynomial(model, x, degree):
    coefs = model.coef()
    alpha = coefs['alpha']
    betas = [coefs[f'beta{i}'] for i in range(1, degree+1)]
    
    X_poly = np.column_stack([x**i for i in range(1, degree + 1)])
    y_pred = alpha + np.sum(X_poly * betas, axis=1)
    
    return y_pred

# Plot all models
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, deg in enumerate(degrees):
    ax = axes[i]
    
    # Plot data
    ax.scatter(mass_std, brain_std, s=100, alpha=0.8, 
              edgecolor='black', linewidth=1.5, zorder=3)
    
    # Plot polynomial fit
    y_plot = predict_polynomial(models[deg], x_plot, deg)
    ax.plot(x_plot, y_plot, 'b-', linewidth=2, alpha=0.7, zorder=2)
    
    # Formatting
    ax.set_xlabel('Body mass (standardized)', fontsize=11)
    ax.set_ylabel('Brain volume (standardized)', fontsize=11)
    ax.set_title(f'Degree {deg} (R² = {r_squared_values[deg]:.3f})\n' + 
                f'{deg + 2} parameters',
                fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-2, 2)
    
    # Add annotation for absurd models
    if deg >= 5:
        ax.text(0.05, 0.95, '⚠️ Overfitting!', 
               transform=ax.transAxes, fontsize=10,
               verticalalignment='top',
               bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

plt.suptitle('Polynomial Overfitting: More Parameters ≠ Better Model', 
            fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\nObservations:")
print("  • Degree 1-2: Reasonable fits")
print("  • Degree 3-4: Starting to wiggle")
print("  • Degree 5-6: Absurdly wiggly, clearly overfitting!")
print("\n  Despite improving R², higher degrees are worse models!")

## The Problem: R² Always Increases!

Let's visualize how R² changes with model complexity:

In [ ]:
# Plot R² vs number of parameters
fig, ax = plt.subplots(figsize=(10, 6))

n_params = [deg + 2 for deg in degrees]  # +2 for alpha and sigma
r2_values = [r_squared_values[deg] for deg in degrees]

ax.plot(n_params, r2_values, 'o-', linewidth=2, markersize=10, 
       color='steelblue', label='R² (training fit)')
ax.axhline(1.0, color='red', linestyle='--', alpha=0.5, label='Perfect fit')

ax.set_xlabel('Number of Parameters', fontsize=12)
ax.set_ylabel('R² (Proportion of Variance Explained)', fontsize=12)
ax.set_title('R² Always Increases with More Parameters\n(But this doesn\'t mean better predictions!)', 
            fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1.05)

# Annotate the problem
ax.annotate('Training fit improves...', 
           xy=(8, r2_values[-1]), xytext=(7, 0.6),
           arrowprops=dict(arrowstyle='->', color='red', lw=2),
           fontsize=11, color='red', fontweight='bold')

ax.annotate('...but predictions\nmay get worse!', 
           xy=(8, r2_values[-1]), xytext=(9, 0.4),
           arrowprops=dict(arrowstyle='->', color='red', lw=2),
           fontsize=11, color='red', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n⚠️ THE PROBLEM:")
print("   R² is NOT a good measure of model quality!")
print("   It always improves with more parameters.")
print("\n✓ WE NEED:")
print("   - Measures that penalize complexity")
print("   - Out-of-sample validation")
print("   - Information criteria (WAIC, LOO)")

---

## Key Lessons

### 1. R² is Misleading
- R² always increases (or stays same) with more parameters
- Perfect R² = 1.0 with N parameters for N data points
- **But** this doesn't mean good predictions!

### 2. Visual Inspection Reveals Absurdity
- Degree 5-6 polynomials are clearly ridiculous
- Wiggly curves that fit noise, not signal
- Would make terrible predictions on new species

### 3. We Need Better Measures
To evaluate models properly, we need:

**Option 1: Information Criteria**
- WAIC, AIC, DIC
- Balance fit vs complexity automatically
- Coming in next sections!

**Option 2: Cross-Validation**
- Test on held-out data
- Direct measure of predictive accuracy
- Gold standard but expensive

**Option 3: Regularization**
- Skeptical priors that resist complexity
- Built into the model itself

### 4. The Overfitting Paradox
> **"Better fit to training data can mean worse predictions!"**

This is why we can't just maximize R² or minimize training error.

---

## Next Steps

In the following notebooks, we'll learn:
1. **Information theory** - Why overfitting happens (entropy, KL divergence)
2. **WAIC** - How to compute and use it for model comparison
3. **Cross-validation** - Direct out-of-sample testing
4. **Regularization** - Using priors to prevent overfitting

These tools will help us choose models wisely!